In [129]:
import os
import glob
from collections import defaultdict
import pandas as pd

### Find all repositories

In [130]:
data_repos = glob.glob('../data/*')
print(data_repos)

['../data/Horizon', '../data/TensorFlow-Examples', '../data/text_classification', '../data/Detectron', '../data/neural-style']


### Save all filenames of each repo in a dictionary

In [131]:
repo_files_dict = defaultdict(list)
for repo in data_repos:
    filenames = glob.glob(repo + '/**/*.py',recursive=True)
    repo_files_dict[repo] = filenames
print(len(repo_files_dict))

5


### Create Pandas dataframe to store repo name and their python files 

In [132]:
repos_df = defaultdict(list)
for repo, filenames in repo_files_dict.items():
    code = ''
    for filename in filenames:
        code += filename + '\n'
        file = open(filename,'r')
        code += file.read()
    repos_df['repo_name'].append(repo)
    repos_df['code'].append(code)

repos_df = pd.DataFrame.from_dict(repos_df)


## Create vocabulary for Encoder
### Refer https://arxiv.org/pdf/1508.07909.pdf

In [138]:
import encoder
import pickle

In [139]:
corpus = ' <\eol>\n'.join(repos_df['code'])
corpus[:1000]

'../data/text_classification/a08_predict_ensemble.py\n# -*- coding: utf-8 -*-\n#prediction using multi-models. take out: create multiple graphs. each graph associate with a session. add logits of models.\n#process--->1.load data(X:list of lint,y:int). 2.create session. 3.feed data. 4.predict\nimport sys\nreload(sys)\nsys.setdefaultencoding(\'utf8\')\nimport tensorflow as tf\nimport numpy as np\nimport os\nfrom a3_entity_network import EntityNetwork\nsys.path.append("..")\nfrom a08_DynamicMemoryNetwork.data_util_zhihu import load_data_predict,load_final_test_data,create_voabulary,create_voabulary_label\nfrom tflearn.data_utils import pad_sequences #to_categorical\nimport codecs\nfrom a08_DynamicMemoryNetwork.a8_dynamic_memory_network import DynamicMemoryNetwork\nfrom p7_TextCNN_model import TextCNN\nfrom p71_TextRCNN_mode2 import TextRCNN\n\n#configuration\nFLAGS=tf.app.flags.FLAGS\ntf.app.flags.DEFINE_integer("num_classes",1999,"number of label")\ntf.app.flags.DEFINE_float("learning_ra

In [143]:
encoder = Encoder(40000, pct_bpe=0.88,ngram_min=1,ngram_max=5)
encoder.fit(corpus.split('\n'))

In [144]:
example = "def trim_vocab(n, vocab):\n# type: (int, Dict[str, int]) -> None\n''' Deletes all pairs below 10 * vocab size to prevent memory problems '''\npair_counts = sorted(vocab.items(), key=lambda p: -p[1])\npairs_to_trim = [pair for pair, count in pair_counts[n:]]\nfor pair in pairs_to_trim:\ndel vocab[pair]"
print(encoder.tokenize(example))
print(next(encoder.transform([example])))
print(next(encoder.inverse_transform(encoder.transform([example]))))

['def', '__sow', 'tri', 'm_v', 'ocab', '__eow', '(', 'n', ',', 'vocab', '):', '#', 'type', ':', '(', 'int', ',', 'dict', '[', 'str', ',', 'int', '])', '->', 'none', "'''", '__sow', 'dele', 'tes', '__eow', 'all', '__sow', 'pairs', '__eow', 'below', '10', '*', 'vocab', 'size', 'to', 'prevent', 'memory', '__sow', 'probl', 'ems', '__eow', "'''", '__sow', 'pair', '_coun', 'ts', '__eow', '=', 'sorted', '(', 'vocab', '.', 'items', '(),', 'key', '=', 'lambda', 'p', ':', '-', 'p', '[', '1', '])', '__sow', 'pairs', '_to_t', 'rim', '__eow', '=', '[', 'pair', 'for', 'pair', ',', 'count', 'in', '__sow', 'pair', '_coun', 'ts', '__eow', '[', 'n', '__sow', ':]]', '__eow', 'for', 'pair', 'in', '__sow', 'pairs', '_to_t', 'rim', '__eow', ':', 'del', 'vocab', '[', 'pair', ']']
[28, 4801, 5167, 15844, 9580, 4800, 5, 190, 3, 673, 20, 7, 136, 9, 5, 115, 3, 452, 12, 164, 3, 115, 73, 270, 45, 797, 4801, 34150, 4975, 4800, 114, 4801, 16229, 4800, 762, 222, 43, 673, 161, 27, 2834, 759, 4801, 39589, 15808, 4800, 

In [142]:
print(len(encoder.bpe_vocab))
encoder.save('../model/bpe.json')

35200
